In [128]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta


In [129]:
yesterday = datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')
url = 'https://puzizig.com/programs/stock_upper_limit/date/' + yesterday


In [130]:
#URL 접근
request = requests.get(url)
soup = BeautifulSoup(request.content,'html.parser')
selected = soup.select('body > div > div > div.col-md-8.mt-3 > div:nth-child(4)')


In [131]:
table_html = selected[0].find_all('div',{'class':'col-md-6 col-lg-3'})
yesterday_price = [table.text[5:].replace(',','') for table in table_html if table.text.startswith('현재가')]

In [132]:
yesterday_code = []
yesterday_company = []
info_html = selected[0].find_all('div',{'class':'d-flex align-items-center mb-2'})
for info in info_html:
    yesterday_code.append(info.find('span',{'class':'badge badge-secondary badge-pill mr-2'}).text)
    yesterday_company.append(info.find('a').text)

In [136]:
for i in reversed(range(len(yesterday_code))):
    if len(yesterday_price[i]) <= 3:
        yesterday_code.pop(i)

In [144]:
def get_yesterday_highest():
        #날짜 설정
        yesterday = datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')
        url = 'https://puzizig.com/programs/stock_upper_limit/date/' + yesterday
        #URL 접근
        request = requests.get(url)
        soup = BeautifulSoup(request.content,'html.parser')
        selected = soup.select('body > div > div > div.col-md-8.mt-3 > div:nth-child(4)')
        #코드 및 종목 찾기
        yesterday_code = []
        yesterday_company = []
        info_html = selected[0].find_all('div',{'class':'d-flex align-items-center mb-2'})
        for info in info_html:
            yesterday_code.append(info.find('span',{'class':'badge badge-secondary badge-pill mr-2'}).text)
            yesterday_company.append(info.find('a').text)
        
        #가격 찾아서 동전주는 깔끔하게 제거
        table_html = selected[0].find_all('div',{'class':'col-md-6 col-lg-3'})
        yesterday_price = [table.text[5:].replace(',','') for table in table_html if table.text.startswith('현재가')]

        for i in reversed(range(len(yesterday_code))):
            if len(yesterday_price[i]) <= 3:
                yesterday_code.pop(i)
        
        print("어제 상한가 종목은: ", yesterday_company)
        return yesterday_code

In [145]:
codes = get_yesterday_highest()
print(codes)

어제 상한가 종목은:  ['까뮤이앤씨', 'SV인베스트먼트', '엠에스씨', '자이글', '한국유니온제약']
['013700', '289080', '009780', '234920', '080720']


In [147]:
import win32com.client
import ctypes
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import time
from PyQt5.QtWidgets import *


g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')


g_objCpTrade.AccountNumber[0]

com_error: (-2147352567, '예외가 발생했습니다.', (0, None, 'Module:CpDib\nObj:CpTdUtil\nFun:AccountNumber\nType:0\nDescription:TradeInit을 하지 않았습니다.', None, 0, -2147418113), None)